In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

#Loading the dataset
df = pd.read_csv("processed_rt_iot2022.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'processed_rt_iot2022.csv'

In [ ]:
#Removed port numbers, since model can be trained according to specific port numbers and decide according to specific ports
df = df.drop(columns=['id.orig_p', 'id.resp_p'])

In [ ]:
#Encoding target variable
le_target = LabelEncoder()
df['Attack_Type'] = le_target.fit_transform(df['Attack_Type'])

#log(1+n) transformation for SVM, to have closer numbers in SVM
log_cols = ['flow_duration', 'active.tot', 'idle.tot']
for col in log_cols:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: np.log1p(x))

In [ ]:
X = df.drop(columns=['Attack_Type'])
y = df['Attack_Type']

In [ ]:
#One-hot encoding
categorical_cols = ['proto', 'service']
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

In [ ]:
#Pipeline for one-hot encoding, standardization, PCA and SVM
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('scaling', StandardScaler()),
    ('pca', PCA(n_components=30, random_state=42)),
    ('svm', SVC(kernel='rbf', class_weight='balanced', random_state=42))
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)


In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['proto', 'service'])])),
                ('scaling', StandardScaler()),
                ('pca', PCA(n_components=30, random_state=42)),
                ('svm', SVC(class_weight='balanced', random_state=42))])

In [ ]:
y_pred = pipeline.predict(X_test)
print("📊 SVM with PCA + Log Transform + One-Hot Encoding:\n")
print(classification_report(y_test, y_pred))

📊 SVM with PCA + Log Transform + One-Hot Encoding:

              precision    recall  f1-score   support

           0       0.96      0.93      0.95      2325
           1       0.75      0.99      0.86       160
           2       1.00      1.00      1.00     28398
           3       1.00      0.99      1.00      1244
           4       0.09      0.91      0.17        11
           5       0.27      0.88      0.41         8
           6       1.00      1.00      1.00       600
           7       0.99      1.00      1.00       301
           8       1.00      0.93      0.96       777
           9       1.00      1.00      1.00       603
          10       0.96      0.95      0.96      2433
          11       0.69      0.89      0.78        76

    accuracy                           0.99     36936
   macro avg       0.81      0.96      0.84     36936
weighted avg       0.99      0.99      0.99     36936



SMOTE Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Update your pipeline to include SMOTE before SVM
pipeline = ImbPipeline(steps=[
    ('preprocessing', preprocessor),
    ('scaling', StandardScaler()),
    ('pca', PCA(n_components=30, random_state=42)),
    ('smote', SMOTE(random_state=42, sampling_strategy={4: 100, 5: 100})),  # Oversample classes 4/5 to 100 samples each
    ('svm', SVC(kernel='rbf', class_weight='balanced', random_state=42))
])

# Fit and evaluate
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      2325
           1       0.75      0.99      0.86       160
           2       1.00      1.00      1.00     28398
           3       1.00      0.99      1.00      1244
           4       0.16      0.91      0.27        11
           5       0.54      0.88      0.67         8
           6       1.00      1.00      1.00       600
           7       0.99      1.00      1.00       301
           8       1.00      0.93      0.96       777
           9       1.00      1.00      1.00       603
          10       0.96      0.95      0.96      2433
          11       0.69      0.89      0.78        76

    accuracy                           0.99     36936
   macro avg       0.84      0.96      0.87     36936
weighted avg       0.99      0.99      0.99     36936



Class-Weighted Undersampling

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import pandas as pd

# 1. Original model (no class weighting)
original_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('scaling', StandardScaler()),
    ('pca', PCA(n_components=30, random_state=42)),
    ('svm', SVC(kernel='rbf', random_state=42))  # No class weights
])
original_pipeline.fit(X_train, y_train)
y_pred_original = original_pipeline.predict(X_test)  # Store original predictions

# 2. Class-weighted model
class_weights = dict(1 / (y_train.value_counts(normalize=True)))
weighted_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('scaling', StandardScaler()),
    ('pca', PCA(n_components=30, random_state=42)),
    ('svm', SVC(kernel='rbf', class_weight=class_weights, random_state=42))
])
weighted_pipeline.fit(X_train, y_train)
y_pred_weighted = weighted_pipeline.predict(X_test)

# 3. Enhanced comparison report
print("=== Performance Comparison ===")
original_report = classification_report(y_test, y_pred_original, output_dict=True)
weighted_report = classification_report(y_test, y_pred_weighted, output_dict=True)

# Create comparison DataFrame
results = []
for cls in sorted(set(y_test)):
    results.append({
        'Class': cls,
        'Original Precision': original_report[str(cls)]['precision'],
        'Weighted Precision': weighted_report[str(cls)]['precision'],
        'Original Recall': original_report[str(cls)]['recall'],
        'Weighted Recall': weighted_report[str(cls)]['recall'],
        'Original F1': original_report[str(cls)]['f1-score'],
        'Weighted F1': weighted_report[str(cls)]['f1-score'],
        'Support': original_report[str(cls)]['support']
    })

print(pd.DataFrame(results).set_index('Class').round(3))

=== Performance Comparison ===
       Original Precision  Weighted Precision  Original Recall  \
Class                                                            
0                   0.933               0.962            0.958   
1                   0.992               0.751            0.781   
2                   1.000               1.000            1.000   
3                   1.000               0.999            0.990   
4                   1.000               0.156            0.818   
5                   0.538               0.163            0.875   
6                   0.998               1.000            1.000   
7                   0.993               0.987            1.000   
8                   0.957               0.999            0.979   
9                   1.000               1.000            0.997   
10                  0.958               0.990            0.955   
11                  1.000               0.893            0.579   

       Weighted Recall  Original F1  Weighte